<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
from pulp import *

Primero definimos agentes y clientes.

In [2]:
#Agentes
Agentes=["Terry","Carle","McClymonds"]
#Clientes
Clientes=["1","2","3"]

La matriz de tiempos, en cada fila el tiempo que gasta en días cada agente con los prespectivos clientes.

In [3]:
tiempos=[ 
    [10,15,9],
    [9,18,5],
    [6,14,3]
]
tiempos=makeDict([Agentes,Clientes],tiempos,0)
tiempos

defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
            {'Terry': defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
                         {'1': 10, '2': 15, '3': 9}),
             'Carle': defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
                         {'1': 9, '2': 18, '3': 5}),
             'McClymonds': defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
                         {'1': 6, '2': 14, '3': 3})})

Definimos el problema de asignación, queremos minimizar tiempo.

In [4]:
probAsign=LpProblem("Problema_de_Asignación",LpMinimize)

Las rutas indican como cada agente atiende a cada cliente.

In [5]:
Rutas = [(p,c) for p in Agentes for c in Clientes]

In [6]:
Rutas

[('Terry', '1'),
 ('Terry', '2'),
 ('Terry', '3'),
 ('Carle', '1'),
 ('Carle', '2'),
 ('Carle', '3'),
 ('McClymonds', '1'),
 ('McClymonds', '2'),
 ('McClymonds', '3')]

Creamos las variables:

In [7]:
variables=LpVariable.dicts("Atención",(Agentes,Clientes),0,None,LpContinuous)

In [8]:
variables

{'Terry': {'1': Atención_Terry_1,
  '2': Atención_Terry_2,
  '3': Atención_Terry_3},
 'Carle': {'1': Atención_Carle_1,
  '2': Atención_Carle_2,
  '3': Atención_Carle_3},
 'McClymonds': {'1': Atención_McClymonds_1,
  '2': Atención_McClymonds_2,
  '3': Atención_McClymonds_3}}

Incluimos la función objetivo:

In [11]:
[variables[p][c]*tiempos[p][c] for (p,c) in Rutas] 

[10*Atención_Terry_1 + 0,
 15*Atención_Terry_2 + 0,
 9*Atención_Terry_3 + 0,
 9*Atención_Carle_1 + 0,
 18*Atención_Carle_2 + 0,
 5*Atención_Carle_3 + 0,
 6*Atención_McClymonds_1 + 0,
 14*Atención_McClymonds_2 + 0,
 3*Atención_McClymonds_3 + 0]

In [12]:
probAsign+= lpSum([variables[p][c]*tiempos[p][c] for (p,c) in Rutas])

Las restricciones

In [16]:
for p in Agentes:
    probAsign +=lpSum([variables[p][c] for c in Clientes])<=1,"Producción_de_%s"%p

In [17]:
for c in Clientes:
    probAsign +=lpSum([variables[p][c] for p in Agentes])==1,"Cliente_%s"%c

In [18]:
probAsign.writeLP

<bound method LpProblem.writeLP of Problema_de_Asignación:
MINIMIZE
9*Atención_Carle_1 + 18*Atención_Carle_2 + 5*Atención_Carle_3 + 6*Atención_McClymonds_1 + 14*Atención_McClymonds_2 + 3*Atención_McClymonds_3 + 10*Atención_Terry_1 + 15*Atención_Terry_2 + 9*Atención_Terry_3 + 0
SUBJECT TO
Producción_de_Terry: Atención_Terry_1 + Atención_Terry_2 + Atención_Terry_3
 <= 1

Producción_de_Carle: Atención_Carle_1 + Atención_Carle_2 + Atención_Carle_3
 <= 1

Producción_de_McClymonds: Atención_McClymonds_1 + Atención_McClymonds_2
 + Atención_McClymonds_3 <= 1

Cliente_1: Atención_Carle_1 + Atención_McClymonds_1 + Atención_Terry_1 = 1

Cliente_2: Atención_Carle_2 + Atención_McClymonds_2 + Atención_Terry_2 = 1

Cliente_3: Atención_Carle_3 + Atención_McClymonds_3 + Atención_Terry_3 = 1

VARIABLES
Atención_Carle_1 Continuous
Atención_Carle_2 Continuous
Atención_Carle_3 Continuous
Atención_McClymonds_1 Continuous
Atención_McClymonds_2 Continuous
Atención_McClymonds_3 Continuous
Atención_Terry_1 Cont

In [19]:
probAsign.solve()

1

In [20]:
for v in probAsign.variables():
    print(v.name,"=",v.varValue)

Atención_Carle_1 = 0.0
Atención_Carle_2 = 0.0
Atención_Carle_3 = 1.0
Atención_McClymonds_1 = 1.0
Atención_McClymonds_2 = 0.0
Atención_McClymonds_3 = 0.0
Atención_Terry_1 = 0.0
Atención_Terry_2 = 1.0
Atención_Terry_3 = 0.0
